In [ ]:
import numpy as np
import matplotlib.pyplot as plot
import heapq


Movimientos = [(-1, -1), (-1, 0), (-1, +1),
               (0,-1), (+1, 0),
               (-1, +1), (0, +1), (+1, +1)]


class Nodo:
    def inicializar(self, position, parent= None, g=0, h=0, risk=0):
        self.position = position,
        self.parent = parent,
        self.g = g,
        self.h = h,
        self.risk = risk,
        self.f = g + h + risk
    
    def menorque(self, other):
        return self.f < other.f
